# 에지 검출과 응용

In [2]:
import cv2
import numpy as np
import sys

## 마스크 기반 에지 검출 - 소벨 마스크

In [15]:
src = cv2.imread("./data/lenna.bmp", cv2.IMREAD_GRAYSCALE)

Mx = np.array(
    [[-1, 0, 1],
     [-2, 0, 2],
     [-1, 0, 1]], dtype= np.float32) # 소벨 마스크 (x방향 미분 마스크)

My = np.array(
    [[-1, -2, -1],
     [0, 0, 0],
     [1, 2, 1]], dtype= np.float32) # 소벨 마스크 (y방향 미분 마스크)

dx = cv2.filter2D(src, -1, Mx, delta=128) # dx : x방향 필터와 원본 이미지 간의 마스크 연산의 결과(미분값)
dy = cv2.filter2D(src, -1, My, delta=128) # dy : y방향 필터와 원본 이미지 간의 마스크 연산의 결과(미분값)

cv2.imshow("src", src)
cv2.imshow("dx", dx)
cv2.imshow("dy", dy)
cv2.waitKey()
cv2.destroyAllWindows()

In [27]:
src = cv2.imread("./data/lenna.bmp", cv2.IMREAD_GRAYSCALE)

# sobel mask 준비 없이, 원본간의 마스크 연산을 한번에..
dx = cv2.Sobel(src, cv2.CV_32FC1, 1, 0) # dx : x방향 필터와 원본 이미지 간의 마스크 연산의 결과(미분값)
dy = cv2.Sobel(src, cv2.CV_32FC1, 0, 1) # dy : y방향 필터와 원본 이미지 간의 마스크 연산의 결과(미분값)
# --> cv2.filter2D()와 다르게 타입이 float32이고, 포화연산이 안되어 있는 상태

fmag = cv2.magnitude(dx, dy) # dx, dy 각각의 미분값을 벡터(그레디언트)의 크기로 반환
mag = np.clip(fmag, 0, 255).astype(np.uint8)

T = 160
_, edge = cv2.threshold(mag, T, 255, cv2.THRESH_BINARY)


cv2.imshow("src", src)
cv2.imshow("mag", mag)
cv2.imshow("edge", edge)
cv2.waitKey()
cv2.destroyAllWindows()

## 캐니 에지 검출기

In [29]:
src = cv2.imread("./data/lenna.bmp", cv2.IMREAD_GRAYSCALE)

# low threshold, high threshold를 변경해 가면서 영상 결과 확인
dst1 = cv2.Canny(src, 50, 100) 
dst2 = cv2.Canny(src, 100, 200)

cv2.imshow("dst1", dst1)
cv2.imshow("dst2", dst2)
cv2.waitKey()
cv2.destroyAllWindows()

## 허프 변환 직선 검출

In [30]:
import math

In [53]:
src = cv2.imread("./data/building.jpg", cv2.IMREAD_GRAYSCALE)


edge = cv2.Canny(src, 100, 200) #(src, low threshold, high threshold)
                                # low:high = 1:2, 1:3

rho = 1
theta = math.pi/180.
threshold = 160 # 축적배열에 직선으로 많이 교차하는 위치의 값이 threshold 이상 넘어갈 때 직선으로 간주
minlineLength = 50
maxlineGap = 5
lines = cv2.HoughLinesP(edge, rho, theta, threshold,minLineLength= minlineLength, maxLineGap=maxlineGap)

dst = cv2.cvtColor(edge,cv2.COLOR_GRAY2BGR) # dst 에 color로 그리기 위해 3 channel 준비
if lines is not None: # 라인 정보를 받았으면
    for i in range(lines.shape[0]):
        pt1 = (lines[i][0][0], lines[i][0][1])
        pt2 = (lines[i][0][2], lines[i][0][3])
        cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)
        
cv2.imshow('src',src)
cv2.imshow('dst',dst)
cv2.waitKey()
cv2.destroyAllWindows()

-- HoughLinesP()의 파라미터를 변경하면서 비교해 볼 수 있는 코드 (by 성록)

In [59]:
src = cv2.imread("./data/building.jpg", cv2.IMREAD_GRAYSCALE)

for i in range(100, 200, 10):
    edge = cv2.Canny(src, 80, 240) #(src, low threshold, high threshold) #보통 1:2~3 비율로 한다. 
    rho = 1
    theta = math.pi/180.
    threshold = i #선 인식 조절 
    minlineLength = 50
    threshold_1 = 120
    minlineLength_1 = i-90
    maxlineGap = 5
    lines = cv2.HoughLinesP(edge, rho, theta, threshold,minLineLength= minlineLength, maxLineGap=maxlineGap)# 라인 데이터 받기
    lines_1 = cv2.HoughLinesP(edge, rho, theta, threshold_1,minLineLength= minlineLength_1, maxLineGap=maxlineGap)# 라인 데이터 받기
    dst = cv2.cvtColor(edge,cv2.COLOR_GRAY2BGR)
    desc = 'HoughLinesP: threshold %d' % (i)
    cv2.putText(dst, desc, (10,590), cv2.FONT_HERSHEY_SIMPLEX, 1.0,200,1,cv2.LINE_AA)


    if lines is not None: 
        for i in range(lines.shape[0]):
            pt1 = (lines[i][0][0], lines[i][0][1])
            pt2 = (lines[i][0][2], lines[i][0][3])
            cv2.line(dst, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)
    dst_1 = cv2.cvtColor(edge,cv2.COLOR_GRAY2BGR)
    desc_1 = 'HoughLinesP: minlineLength %d' % (minlineLength_1)
    cv2.putText(dst_1, desc_1, (10,590), cv2.FONT_HERSHEY_SIMPLEX, 1.0,200,1,cv2.LINE_AA)
    if lines_1 is not None: 
        for i in range(lines_1.shape[0]):
            pt1 = (lines_1[i][0][0], lines_1[i][0][1])
            pt2 = (lines_1[i][0][2], lines_1[i][0][3])
            cv2.line(dst_1, pt1, pt2, (0, 0, 255), 2, cv2.LINE_AA)
            
    cv2.imshow('dst_1',dst_1)
    cv2.imshow('dst',dst)
    cv2.waitKey()
cv2.destroyAllWindows()


## 허프 변환 원 검출

In [80]:
src = cv2.imread("./data/coins.png", cv2.IMREAD_GRAYSCALE)

blurred = cv2.blur(src, (3, 3))

circles = cv2.HoughCircles(blurred, cv2.HOUGH_GRADIENT, dp=1, minDist=50, 
                 param1=150, param2=30)

dst = cv2.cvtColor(src, cv2.COLOR_GRAY2BGR)

if circles is not None:
    for i in range(circles.shape[1]):
        cx, cy, radius = circles[0][i]
        cv2.circle(dst, (np.uint32(cx), np.uint32(cy)), np.uint32(radius), (0, 0, 255), 2)

cv2.imshow('src',src)
cv2.imshow('dst',dst)
cv2.imshow('blurred',blurred)
cv2.waitKey()
cv2.destroyAllWindows()